This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/"))
print(os.listdir("../working/"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

['learn-together']
['.ipynb_checkpoints', '__notebook_source__.ipynb']
/kaggle/input/learn-together/test.csv
/kaggle/input/learn-together/train.csv
/kaggle/input/learn-together/sample_submission.csv


In [2]:
#load data, I had an issue with the data and hacked the data into the kernel
#dftrain=pd.read_csv('/kaggle/input/train.csv')
#dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [ ]:
#file shape
print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

In [ ]:
#features types
print(dftrain.dtypes)
print(dftest.dtypes)

In [ ]:
#validate files - nans per feature
print(dftrain.isnull().sum(axis = 0))
print(dftest.isnull().sum(axis = 0))

In [ ]:
#validate files - no rows with all zeros
print(dftrain[dftrain.drop(['Id','Cover_Type'], axis=1).eq(0).all(1)].empty)
print(dftest[dftest.drop('Id', axis=1).eq(0).all(1)].empty)

In [3]:
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

In [4]:
#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

In [5]:
#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173                      

In [ ]:
#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

In [6]:
# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

In [8]:
#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [9]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

num_features = 54

model = Sequential()
model.add(Dense(units=num_features, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])

Using TensorFlow backend.


In [10]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 3s 212us/sample - loss: 1.6067 - acc: 0.2730 - val_loss: 1.1574 - val_acc: 0.5241
Epoch 2/250
12096/12096 [==============================] - 2s 152us/sample - loss: 1.0566 - acc: 0.5370 - val_loss: 0.9281 - val_acc: 0.5880
Epoch 3/250
12096/12096 [==============================] - 2s 150us/sample - loss: 0.9555 - acc: 0.5770 - val_loss: 0.8757 - val_acc: 0.6323
Epoch 4/250
12096/12096 [==============================] - 2s 161us/sample - loss: 0.9231 - acc: 0.5933 - val_loss: 0.8073 - val_acc: 0.6376
Epoch 5/250
12096/12096 [==============================] - 2s 150us/sample - loss: 0.8859 - acc: 0.6088 - val_loss: 0.7968 - val_acc: 0.6505
Epoch 6/250
12096/12096 [==============================] - 2s 161us/sample - loss: 0.8823 - acc: 0.6173 - val_loss: 0.8012 - val_acc: 0.6481
Epoch 7/250
12096/12096 [==============================] - 2s 150us/sample - loss: 0.8747 - acc: 0.6176 -

12096/12096 [==============================] - 2s 153us/sample - loss: 0.7110 - acc: 0.6965 - val_loss: 0.7151 - val_acc: 0.6862
Epoch 59/250
12096/12096 [==============================] - 2s 157us/sample - loss: 0.7272 - acc: 0.6844 - val_loss: 0.6549 - val_acc: 0.7153
Epoch 60/250
12096/12096 [==============================] - 2s 165us/sample - loss: 0.7138 - acc: 0.6925 - val_loss: 0.6679 - val_acc: 0.7315
Epoch 61/250
12096/12096 [==============================] - 2s 159us/sample - loss: 0.7097 - acc: 0.6968 - val_loss: 0.6929 - val_acc: 0.7037
Epoch 62/250
12096/12096 [==============================] - 2s 158us/sample - loss: 0.7086 - acc: 0.7010 - val_loss: 0.6515 - val_acc: 0.7378
Epoch 63/250
12096/12096 [==============================] - 2s 155us/sample - loss: 0.7071 - acc: 0.7022 - val_loss: 0.6538 - val_acc: 0.7298
Epoch 64/250
12096/12096 [==============================] - 2s 168us/sample - loss: 0.7069 - acc: 0.7007 - val_loss: 0.6823 - val_acc: 0.7226
Epoch 65/250
12096/

12096/12096 [==============================] - 2s 172us/sample - loss: 0.6473 - acc: 0.7359 - val_loss: 0.6408 - val_acc: 0.7470
Epoch 116/250
12096/12096 [==============================] - 2s 163us/sample - loss: 0.6747 - acc: 0.7232 - val_loss: 0.6727 - val_acc: 0.7335
Epoch 117/250
12096/12096 [==============================] - 2s 170us/sample - loss: 0.6701 - acc: 0.7228 - val_loss: 0.6714 - val_acc: 0.7285
Epoch 118/250
12096/12096 [==============================] - 2s 159us/sample - loss: 0.6653 - acc: 0.7328 - val_loss: 0.6722 - val_acc: 0.7285
Epoch 119/250
12096/12096 [==============================] - 2s 174us/sample - loss: 0.6657 - acc: 0.7342 - val_loss: 0.6242 - val_acc: 0.7483
Epoch 120/250
12096/12096 [==============================] - 2s 163us/sample - loss: 0.6644 - acc: 0.7318 - val_loss: 0.6443 - val_acc: 0.7371
Epoch 121/250
12096/12096 [==============================] - 2s 165us/sample - loss: 0.6636 - acc: 0.7267 - val_loss: 0.6737 - val_acc: 0.7146
Epoch 122/250

12096/12096 [==============================] - 2s 164us/sample - loss: 0.6422 - acc: 0.7434 - val_loss: 0.6482 - val_acc: 0.7493
Epoch 173/250
12096/12096 [==============================] - 2s 163us/sample - loss: 0.6383 - acc: 0.7427 - val_loss: 0.6721 - val_acc: 0.7298
Epoch 174/250
12096/12096 [==============================] - 2s 167us/sample - loss: 0.6464 - acc: 0.7438 - val_loss: 0.6349 - val_acc: 0.7361
Epoch 175/250
12096/12096 [==============================] - 2s 168us/sample - loss: 0.6472 - acc: 0.7378 - val_loss: 0.6048 - val_acc: 0.7513
Epoch 176/250
12096/12096 [==============================] - 2s 163us/sample - loss: 0.6330 - acc: 0.7430 - val_loss: 0.6185 - val_acc: 0.7480
Epoch 177/250
12096/12096 [==============================] - 2s 161us/sample - loss: 0.6369 - acc: 0.7439 - val_loss: 0.6263 - val_acc: 0.7543
Epoch 178/250
12096/12096 [==============================] - 2s 160us/sample - loss: 0.6452 - acc: 0.7418 - val_loss: 0.6283 - val_acc: 0.7526
Epoch 179/250

12096/12096 [==============================] - 2s 149us/sample - loss: 0.6363 - acc: 0.7426 - val_loss: 0.5958 - val_acc: 0.7536
Epoch 230/250
12096/12096 [==============================] - 2s 148us/sample - loss: 0.6068 - acc: 0.7547 - val_loss: 0.6218 - val_acc: 0.7467
Epoch 231/250
12096/12096 [==============================] - 2s 151us/sample - loss: 0.6226 - acc: 0.7470 - val_loss: 0.6012 - val_acc: 0.7622
Epoch 232/250
12096/12096 [==============================] - 2s 158us/sample - loss: 0.6206 - acc: 0.7503 - val_loss: 0.6083 - val_acc: 0.7507
Epoch 233/250
12096/12096 [==============================] - 2s 161us/sample - loss: 0.6235 - acc: 0.7479 - val_loss: 0.5998 - val_acc: 0.7576
Epoch 234/250
12096/12096 [==============================] - 2s 161us/sample - loss: 0.6284 - acc: 0.7474 - val_loss: 0.6053 - val_acc: 0.7652
Epoch 235/250
12096/12096 [==============================] - 2s 156us/sample - loss: 0.6324 - acc: 0.7402 - val_loss: 0.5935 - val_acc: 0.7530
Epoch 236/250

In [11]:
# Predict!!
y_predict = model.predict(x_predict)

In [ ]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

In [12]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

In [13]:
#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv